# Evolution of agents - RAG

Retrieval-Augmented Generation (RAG) is an AI technique that combines information retrieval from external sources with generative models to produce more accurate and contextually relevant responses.

<img src="./images/rag.png" alt="RAG" style="max-height: 300px;" />


In [ ]:
# Example: Inference using Semantic Kernel
import datetime
from typing import List
from semantic_kernel.connectors.ai.azure_ai_inference import (
    AzureAIInferenceChatCompletion,
    AzureAIInferenceChatPromptExecutionSettings,
)
from semantic_kernel.contents import ChatHistory
from azure.ai.inference.aio import ChatCompletionsClient
import os
from setup import get_credentials
import logging

foundry_name = os.environ.get("AZURE_AI_FOUNDRY_NAME", None)

creds = get_credentials()

client = ChatCompletionsClient(
    endpoint=f"https://{foundry_name}.services.ai.azure.com/models",
    credential=creds,
    credential_scopes=["https://cognitiveservices.azure.com/.default"],
)

request_settings = AzureAIInferenceChatPromptExecutionSettings()


async def chat(
    input: str,
    system_message: str = "You are a helpful assistant. You try to rhyme whenever you can.",
    model: str = "gpt-35-turbo",
    other_messages: List[str] = [],
):
    chat_completion_service = AzureAIInferenceChatCompletion(
        ai_model_id=model,
        client=client,
    )

    chat_history = ChatHistory(system_message=system_message)
    for message in other_messages:
        chat_history.add_system_message(message)

    chat_history.add_user_message(input)
    logging.info(f"User:> {input}")

    # Get the chat message content from the chat completion service.
    response = await chat_completion_service.get_chat_message_content(
        chat_history=chat_history,
        settings=request_settings,
    )
    if response:
        return f"LLM:> {response}"

In [ ]:
user_input = "what date is today?"
await chat(user_input)

In [ ]:
await chat(
    user_input,
    model="gpt-4.1",
    other_messages=[f"Today is {datetime.datetime.now().strftime('%Y-%m-%d')}"],
)

In [ ]:
# read BUGBUSTER.md from a file
bugbuster_instructions = ""
with open("docs/BUGBUSTER.md", "r") as file:
    bugbuster_instructions = file.read()

user_input = "what are the mixing instructions for BUGBUSTER™ ULTRA INSECT ELIMINATOR?"
system_message = """
You are a helpful assistant that provides accurate and concise information for user queries. 
Use emojis to make content more readable.

Do not guess. 
If you don't have detailed information, reply with 'I'm sorry, I don't have that information.'
"""
await chat(
    input=user_input,
    system_message=system_message,
    other_messages=[
        bugbuster_instructions,
        f"Today is {datetime.datetime.now().strftime('%Y-%m-%d')}",
    ],
)